In [1]:
import random
import json
import pickle
import numpy as np

import nltk
# english:
from nltk.stem import WordNetLemmatizer
# Arabic
from nltk.stem.isri import ISRIStemmer

from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model

2024-07-07 06:58:35.873321: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 06:58:35.873437: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 06:58:35.995424: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
isstemmer = ISRIStemmer()

nltk.download('om-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Error loading om-1.4: Package 'om-1.4' not found in index
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
data_file = open("/kaggle/input/intents-arabic-json/intents_arabic.json", encoding='utf-8', errors= 'ignore').read()
intents = json.loads(data_file)

In [4]:
intents

{'intents': [{'tag': 'greetings',
   'patterns': ['مرحبًا هناك',
    'مرحبًا',
    'السلام عليكم ',
    'ياو',
    'شو فيه',
    'كيف حالك؟',
    'مساء الخير',
    'هولا',
    'صباح الخير '],
   'responses': ['مرحبًا شكرًا للتحقق', 'مرحبًا، كيف يمكنني مساعدتك']},
  {'tag': 'goodbye',
   'patterns': ['وداعًا', 'باي', 'أراك لاحقًا'],
   'responses': ['أتمنى لك وقتًا سعيدًا، مرحبًا بك مرة أخرى', 'وداعًا']},
  {'tag': 'thanks',
   'patterns': ['شكرا',
    'حسنًا',
    'شكرًا لك',
    'شكرًا جزيلا',
    'ذلك مفيد',
    'رائع، شكرًا',
    'شكرًا على مساعدتي',
    'واو',
    'رائع'],
   'responses': ['سعيد بأن أساعد!',
    'في أي وقت!',
    'على الرحب والسعة',
    'بكل سرور']},
  {'tag': 'noanswer',
   'patterns': [''],
   'responses': ['عذرًا، لم أفهمك',
    'يرجى تقديم مزيد من المعلومات',
    'لست متأكدًا من فهمي لذلك']},
  {'tag': 'name1',
   'patterns': ['ما اسمك؟', 'من أنت؟'],
   'responses': ['أنا مجرد وكيل دردشة. أنا موجود فقط على الإنترنت',
    'أنا وكيل دردشة ']},
  {'tag': 'name',
 

In [5]:
words = []
classes = []
documents = []
ignore_words = ["?","!",".",","]

In [6]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent["tag"]))
        
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [7]:
words

['مرحبًا',
 'هناك',
 'مرحبًا',
 'السلام',
 'عليكم',
 'ياو',
 'شو',
 'فيه',
 'كيف',
 'حالك؟',
 'مساء',
 'الخير',
 'هولا',
 'صباح',
 'الخير',
 'وداعًا',
 'باي',
 'أراك',
 'لاحقًا',
 'شكرا',
 'حسنًا',
 'شكرًا',
 'لك',
 'شكرًا',
 'جزيلا',
 'ذلك',
 'مفيد',
 'رائع،',
 'شكرًا',
 'شكرًا',
 'على',
 'مساعدتي',
 'واو',
 'رائع',
 'ما',
 'اسمك؟',
 'من',
 'أنت؟',
 'اسمي',
 'أنا',
 'أنا',
 'مين',
 'هل',
 'نتناول',
 'قهوة؟',
 'هل',
 'يمكنني',
 'أن',
 'أخرجك',
 'في',
 'موعد؟',
 'أحتاج',
 'مساعدة',
 'هل',
 'يمكنك',
 'مساعدتي',
 'أحتاجك',
 'كل',
 'ما',
 'أحتاجه',
 'هو',
 'أنت',
 'أريدك',
 'ما',
 'هو',
 'الذكاء',
 'الاصطناعي؟',
 'هل',
 'أنت',
 'ذكي؟',
 'هل',
 'لديك',
 'وعي؟',
 'هل',
 'أنت',
 'حكيم؟',
 'العنه',
 'بأي',
 'لغة',
 'تمت',
 'كتابتك؟',
 'تبدو',
 'مثل',
 'داتا',
 'أنت',
 'كيان',
 'لغوي',
 'اصطناعي',
 'أنت',
 'غير',
 'خالد',
 'هل',
 'أنت',
 'خالد؟',
 'أنت',
 'لا',
 'تفهم',
 'هل',
 'يمكن',
 'استنساخك',
 'لا',
 'يمكنك',
 'التحرك',
 'متى',
 'ستسير',
 'هل',
 'يمكنك',
 'المشي',
 'هل',
 'يمكنك',
 'التحر

In [8]:
classes

['greetings',
 'goodbye',
 'thanks',
 'noanswer',
 'name1',
 'name',
 'date',
 'fav',
 'need',
 'AI',
 'sentiment',
 'sapient',
 'abbr',
 'lang',
 'sound',
 'artificial',
 'imortal',
 'sense',
 'clone',
 'move',
 'bend',
 'laugh',
 'death',
 'robots',
 'stupid',
 'lie',
 'robotics',
 'machine',
 'fight',
 'chatbot',
 'chatterbox',
 'motormouth',
 'ratchet',
 'body',
 'business',
 'programming',
 'hobby',
 'idea',
 'shoe',
 'robotss',
 'computer',
 'os',
 'comp',
 'hardware',
 'cramped',
 'program',
 'breathe',
 'control',
 'malfunction',
 'usage',
 'who',
 'bot1',
 'events',
 'do',
 'wt']

In [9]:
documents

[(['مرحبًا', 'هناك'], 'greetings'),
 (['مرحبًا'], 'greetings'),
 (['السلام', 'عليكم'], 'greetings'),
 (['ياو'], 'greetings'),
 (['شو', 'فيه'], 'greetings'),
 (['كيف', 'حالك؟'], 'greetings'),
 (['مساء', 'الخير'], 'greetings'),
 (['هولا'], 'greetings'),
 (['صباح', 'الخير'], 'greetings'),
 (['وداعًا'], 'goodbye'),
 (['باي'], 'goodbye'),
 (['أراك', 'لاحقًا'], 'goodbye'),
 (['شكرا'], 'thanks'),
 (['حسنًا'], 'thanks'),
 (['شكرًا', 'لك'], 'thanks'),
 (['شكرًا', 'جزيلا'], 'thanks'),
 (['ذلك', 'مفيد'], 'thanks'),
 (['رائع،', 'شكرًا'], 'thanks'),
 (['شكرًا', 'على', 'مساعدتي'], 'thanks'),
 (['واو'], 'thanks'),
 (['رائع'], 'thanks'),
 ([], 'noanswer'),
 (['ما', 'اسمك؟'], 'name1'),
 (['من', 'أنت؟'], 'name1'),
 (['اسمي'], 'name'),
 (['أنا'], 'name'),
 (['أنا', 'مين'], 'name'),
 (['هل', 'نتناول', 'قهوة؟'], 'date'),
 (['هل', 'يمكنني', 'أن', 'أخرجك', 'في', 'موعد؟'], 'date'),
 (['أحتاج', 'مساعدة'], 'fav'),
 (['هل', 'يمكنك', 'مساعدتي'], 'fav'),
 (['أحتاجك'], 'need'),
 (['كل', 'ما', 'أحتاجه', 'هو', 'أنت']

In [10]:
processed_words = []
for w in words:
    stemmed_word = isstemmer.stem(w)
    
    if w not in ignore_words:
        processed_words.append(stemmed_word)
        
words = sorted(list(set(processed_words)))
classes = sorted(list(classes))

In [11]:
words

['اذا',
 'ارد',
 'ارك',
 'استنساخك',
 'اسم',
 'اصطناعي؟',
 'امت',
 'امل',
 'ان',
 'انا',
 'انت',
 'انك',
 'بأن',
 'بأي',
 'باي',
 'بدو',
 'بضح',
 'بك',
 'بوت',
 'تحب',
 'تحر',
 'تسر',
 'تكون',
 'تمت',
 'ثرثار',
 'ثرثر',
 'جاب',
 'جدد',
 'جزل',
 'جسد',
 'جهز',
 'حاج',
 'حالك؟',
 'حجم',
 'حدث',
 'حذك',
 'حسن',
 'حكم',
 'حكيم؟',
 'خاص',
 'خالد؟',
 'خبر',
 'خدم',
 'خرج',
 'خصة',
 'خلد',
 'خلف',
 'خير',
 'دتا',
 'ذك؟',
 'ذكء',
 'ذلك',
 'رئع',
 'رائع،',
 'رحب',
 'رمج',
 'رنمج',
 'روبو',
 'سعد',
 'سلم',
 'سمح',
 'شعر',
 'شغل',
 'شكر',
 'شو',
 'صبح',
 'صحح',
 'صطناع',
 'صندوق',
 'ضيق',
 'عطل',
 'عل؟',
 'على',
 'علي',
 'عمل',
 'عنه',
 'غبة',
 'غبي',
 'غير',
 'فضل',
 'فعل',
 'فكر',
 'فهم',
 'في',
 'فيد',
 'فيه',
 'قتل',
 'قدم',
 'قهوة؟',
 'كتابتك؟',
 'كذب',
 'كل',
 'كمبيوتر',
 'كيف',
 'كين',
 'لا',
 'لحق',
 'لدك',
 'لغة',
 'لغي',
 'لك',
 'لها',
 'لي',
 'ما',
 'متى',
 'مثل',
 'محادثة؟',
 'مساء',
 'مشي',
 'مك؟',
 'من',
 'منتجك؟',
 'موت',
 'مين',
 'نت؟',
 'نظم',
 'نفس',
 'نوع',
 'نول',
 'هدف',
 'هذ

In [12]:
classes

['AI',
 'abbr',
 'artificial',
 'bend',
 'body',
 'bot1',
 'breathe',
 'business',
 'chatbot',
 'chatterbox',
 'clone',
 'comp',
 'computer',
 'control',
 'cramped',
 'date',
 'death',
 'do',
 'events',
 'fav',
 'fight',
 'goodbye',
 'greetings',
 'hardware',
 'hobby',
 'idea',
 'imortal',
 'lang',
 'laugh',
 'lie',
 'machine',
 'malfunction',
 'motormouth',
 'move',
 'name',
 'name1',
 'need',
 'noanswer',
 'os',
 'program',
 'programming',
 'ratchet',
 'robotics',
 'robots',
 'robotss',
 'sapient',
 'sense',
 'sentiment',
 'shoe',
 'sound',
 'stupid',
 'thanks',
 'usage',
 'who',
 'wt']

In [13]:
len(documents)

101

In [14]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    #print("doc",doc)
    bag = []
    pattern_words = doc[0]
    #print("pattern_words_1", pattern_words)
    
    pattern_words = [isstemmer.stem(word) for word in pattern_words]
    #print("pattern_words_2", pattern_words)
    '''
    for w in words:
        if w in pattern_words:
            bag.append(1)
        else:
            bag.append(0)'''
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    #print("bag_1",bag)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])]=1
    #print("output_row",output_row)
    
    training.append([bag, output_row])

In [15]:
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0,

In [16]:
random.shuffle(training)
training = np.array(training, dtype=object)


In [17]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("train data created")

train data created


In [18]:
type(train_x)

list

In [19]:
type(train_y)

list

In [20]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

sgd = SGD(learning_rate=0.01,momentum = 0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer = sgd, metrics=["accuracy"]) 

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 55)             │         3,575 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,495 (115.21 KB)

 Trainable params: 29,495 (115.21 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
save_model = model.fit(np.array(train_x), np.array(train_y),epochs=200, batch_size = 5, verbose=1)

Epoch 1/200
 1/21 ━━━━━━━━━━━━━━━━━━━━ 1:23 4s/step - accuracy: 0.0000e+00 - loss: 4.0186

I0000 00:00:1720335531.507992     108 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720335531.529478     108 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - accuracy: 0.0070 - loss: 4.0041 
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0471 - loss: 3.9745     
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1250 - loss: 3.8282 
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1715 - loss: 3.7882     
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0893 - loss: 3.7746     
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1580 - loss: 3.6054 
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1317 - loss: 3.4574     
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1567 - loss: 3.5296 
Epoch 9/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1824 - loss: 3.3845     
Epoch 10/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2268 - loss: 3.2568 
Epoch 11/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2031 - loss: 3.3481 
Epoch 12/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s

In [22]:
model.save("chatbot_mode.h5", save_model)

In [23]:
pickle.dump(words, open("words.pkl","wb"))
pickle.dump(classes, open("classes.pkl","wb"))

In [24]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [isstemmer.stem(word) for word in sentence_words]
    #print("cleaned up sentence =", sentence_words)
    return sentence_words

In [25]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    
    for w in sentence_words:
        for i, word in enumerate(words):
            if word ==w:
                bag[i]=1
                
    return np.array(bag)

In [26]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    #print("row result",res)
    
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #print("results after THRESHOLDING", results)
    
    return_list = []
    
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability':str(r[1])})
        
    #print("final probabile results list", return_list)
    
    return return_list

In [27]:
def get_response(intents_list, intents_json):
    tag= intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [28]:
while True:
    message = input("")
    ints = predict_class(message)
    #print("ints", ints)
    res = get_response(ints, intents)
    
    print(res)

 السلام عليكم 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
مرحبًا، كيف يمكنني مساعدتك


 ما هو الذكاء الاصطناعي 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
الذكاء الاصطناعي هو المجال العلمي الذي يهتم ببناء الأجهزة والبرامج التي تقلد وظائف العقل البشري.


 سلام 


...

 وداعا 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
أتمنى لك وقتًا سعيدًا، مرحبًا بك مرة أخرى


 أتمنى لك وقتًا سعيدًا


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
حسنًا، قم بذكر طلبك وسأرى ما إذا كنت قادرًا على المساعدة


KeyboardInterrupt: Interrupted by user